In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.features import GeoJsonTooltip

# Load the shapefile
gdf = gpd.read_file('/content/LSOA/GM_LSOAs.shp')

# Load the CSV file
csv_data = pd.read_csv('/content/census2021-ts044-lsoa.csv')

# Ensure the data types match for merging
csv_data['geography code'] = csv_data['geography code'].astype(str)
gdf['code'] = gdf['code'].astype(str)

# Merge the GeoDataFrame with the DataFrame on the specified columns
merged_gdf = gdf.merge(csv_data, left_on='code', right_on='geography code', how='inner')

# Set the CRS to WGS84
merged_gdf = merged_gdf.to_crs(epsg=4326)

# Convert the merged GeoDataFrame to GeoJSON
geojson_data = merged_gdf.to_json()

# Create a Folium map at a suitable location and zoom level with no tiles
center = [merged_gdf.geometry.centroid.y.mean(), merged_gdf.geometry.centroid.x.mean()]
map_folium = folium.Map(location=center, zoom_start=12, tiles=None)  # tiles set to None

# Define the fields you want to show on hover from the CSV
tooltip_fields = [
    'Accommodation type: Total: All households',
    'Accommodation type: Detached',
    'Accommodation type: Semi-detached',
    'Accommodation type: Terraced',
    'Accommodation type: In a purpose-built block of flats or tenement',
    'Accommodation type: Part of a converted or shared house, including bedsits',
    'Accommodation type: Part of another converted building, for example, former school, church or warehouse',
    'Accommodation type: In a commercial building, for example, in an office building, hotel or over a shop',
    'Accommodation type: A caravan or other mobile or temporary structure'
]
aliases = [f'{field}:' for field in tooltip_fields]

# Add GeoJSON to the map with a tooltip
folium.GeoJson(
    geojson_data,
    tooltip=GeoJsonTooltip(
        fields=tooltip_fields,
        aliases=aliases,
        localize=True
    ),
    style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 0.5}
).add_to(map_folium)

# Display the map in the notebook for verification
display(map_folium)

# Save the map to an HTML file
map_folium.save('output_map1.html')


<ipython-input-1-b8bbd1d701f1>:26: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [merged_gdf.geometry.centroid.y.mean(), merged_gdf.geometry.centroid.x.mean()]


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/TheMill/Collective_Census_2021_folder/ <dataset file path>')

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
from folium.features import GeoJsonTooltip

def create_map(shapefile_path, data_csv_path, merge_on_shapefile, merge_on_csv, exclude_columns=None):
    # Load the shapefile
    gdf = gpd.read_file(shapefile_path)

    # Load the CSV file
    csv_data = pd.read_csv(data_csv_path)

    # Ensure the data types match for merging
    csv_data[merge_on_csv] = csv_data[merge_on_csv].astype(str)
    gdf[merge_on_shapefile] = gdf[merge_on_shapefile].astype(str)

    # Merge the GeoDataFrame with the DataFrame on the specified columns
    merged_gdf = gdf.merge(csv_data, left_on=merge_on_shapefile, right_on=merge_on_csv, how='inner')

    # Set the CRS to WGS84
    merged_gdf = merged_gdf.to_crs(epsg=4326)

    # Convert the merged GeoDataFrame to GeoJSON
    geojson_data = merged_gdf.to_json()

    # Create a Folium map at a suitable location and zoom level with no tiles
    center = [merged_gdf.geometry.centroid.y.mean(), merged_gdf.geometry.centroid.x.mean()]
    map_folium = folium.Map(location=center, zoom_start=12, tiles=None)

    # Determine tooltip fields based on available columns and exclude specified ones
    if exclude_columns is not None:
        tooltip_fields = [col for col in csv_data.columns if col not in exclude_columns]
    else:
        tooltip_fields = list(csv_data.columns)

    aliases = [f'{field}:' for field in tooltip_fields]

    # Add GeoJSON to the map with a tooltip
    folium.GeoJson(
        geojson_data,
        tooltip=GeoJsonTooltip(
            fields=tooltip_fields,
            aliases=aliases,
            localize=True
        ),
        style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 0.5}
    ).add_to(map_folium)

    # Display the map in the notebook for verification
    display(map_folium)

    # Save the map to an HTML file
    map_folium.save('output_map.html')

# Example usage:
shapefile_path = '/content/LSOA/GM_LSOAs.shp'
data_csv_housing = '/content/census2021-ts044-lsoa.csv'

exclude_columns = ['date','geography code', 'geography']  # Columns to exclude from tooltips
create_map(shapefile_path, data_csv_housing, 'code', 'geography code', exclude_columns)


<ipython-input-3-f7640558cd05>:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [merged_gdf.geometry.centroid.y.mean(), merged_gdf.geometry.centroid.x.mean()]


In [ ]:
import geopandas as gpd
import pandas as pd
import json
import folium
from folium import IFrame
from branca.element import Template, MacroElement

def create_interactive_map(shapefile_path, data_csv_path, merge_on_shapefile, merge_on_csv, exclude_columns=None):
    # Load the shapefile
    gdf = gpd.read_file(shapefile_path)

    # Load the CSV file
    csv_data = pd.read_csv(data_csv_path)

    # Ensure the data types match for merging
    csv_data[merge_on_csv] = csv_data[merge_on_csv].astype(str)
    gdf[merge_on_shapefile] = gdf[merge_on_shapefile].astype(str)

    # Merge the GeoDataFrame with the DataFrame on the specified columns
    merged_gdf = gdf.merge(csv_data, left_on=merge_on_shapefile, right_on=merge_on_csv, how='inner')

    # Set the CRS to WGS84
    merged_gdf = merged_gdf.to_crs(epsg=4326)

    # Convert the merged GeoDataFrame to GeoJSON
    geojson_str = merged_gdf.to_json()
    geojson_data = json.loads(geojson_str)

    # Create a Folium map at a suitable location and zoom level
    center = [merged_gdf.geometry.centroid.y.mean(), merged_gdf.geometry.centroid.x.mean()]
    map_folium = folium.Map(location=center, zoom_start=12, tiles='cartodbpositron')

    # Define the tooltip fields dynamically from the columns of the CSV
    if exclude_columns:
        tooltip_fields = [col for col in csv_data.columns if col not in exclude_columns]
    else:
        tooltip_fields = list(csv_data.columns)

    folium.GeoJson(
        geojson_data,
        style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 0.5},
        tooltip=folium.features.GeoJsonTooltip(fields=tooltip_fields),
    ).add_to(map_folium)

    # Define a template string for Chart.js integration
    template = """
    {% macro html(this, kwargs) %}
    <!doctype html>
    <html lang="en">
    <head>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <title>Interactive Map</title>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
        <style>
            #chart-container { width: 650px; height: 400px; }
        </style>
    </head>
    <body>
        <div id="chart-container">
            <canvas id="myChart"></canvas>
        </div>
        <script>
            var ctx = document.getElementById('myChart').getContext('2d');
            var myChart = new Chart(ctx, {
                type: 'bar',
                data: {
                    labels: [],
                    datasets: [{
                        label: 'Data Value',
                        data: [],
                        backgroundColor: 'rgba(255, 99, 132, 0.2)',
                        borderColor: 'rgba(255, 99, 132, 1)',
                        borderWidth: 1
                    }]
                },
                options: {
                    scales: {
                        y: {
                            beginAtZero: true
                        }
                    }
                }
            });

            function updateChart(feature) {
                myChart.data.labels = [];
                myChart.data.datasets.forEach((dataset) => {
                    dataset.data = [];
                });
                Object.keys(feature.properties).forEach(function(key) {
                    myChart.data.labels.push(key);
                    myChart.data.datasets.forEach((dataset) => {
                        dataset.data.push(feature.properties[key]);
                    });
                });
                myChart.update();
            }

            document.querySelectorAll('.folium-clickable').forEach(function(item) {
                item.addEventListener('click', function () {
                    var feature = JSON.parse(this.getAttribute('data-geojson'));
                    updateChart(feature);
                });
            });
        </script>
    </body>
    </html>
    {% endmacro %}
    """
    macro = MacroElement()
    macro._template = Template(template)
    map_folium.save('interactive_map.html')

# Example usage:
# Example usage:
shapefile_path = '/content/LSOA/GM_LSOAs.shp'
data_csv_housing = '/content/census2021-ts044-lsoa.csv'
exclude_columns = ['date','geography code', 'geography']  # Columns to exclude from tooltips
create_map(shapefile_path, data_csv_housing, 'code', 'geography code', exclude_columns)


<ipython-input-3-f7640558cd05>:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [merged_gdf.geometry.centroid.y.mean(), merged_gdf.geometry.centroid.x.mean()]
